In [21]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
p2 = pd.read_csv('p2_data.csv',encoding='latin-1')

In [69]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords and pet names
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')][1:]

In [97]:
#spliting the data from the begining so I could evaluate the model
Description = p2['ItemDescription'][:10000]
Description2 = p2['ItemDescription'][10000:]
Flag = p2['PreventiveFlag'][:10000]
Flag2 = p2['PreventiveFlag'][10000:]

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

In [79]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(Description)
print(len(bow_transformer.vocabulary_))

6029


In [80]:
description_bow = bow_transformer.transform(Description)

In [81]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(description_bow)

In [82]:
description_tfidf = tfidf_transformer.transform(description_bow1)
print(description_tfidf.shape)

(10000, 6029)


In [83]:
from sklearn.naive_bayes import MultinomialNB

In [84]:
flag_detect_model = MultinomialNB().fit(description_tfidf, Flag)

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

des_train, des_test, flg_train, flg_test = \
train_test_split(Description, Flag, test_size=0.2)

print(len(des_train), len(des_test), len(des_train) + len(des_test))

8000 2000 10000


In [86]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [87]:
pipeline.fit(des_train,flg_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_process at 0x0000028E177562F0>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocesso...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [88]:
predictions = pipeline.predict(des_test)

In [91]:
print(classification_report(predictions,flg_test))

              precision    recall  f1-score   support

         0.0       0.97      0.93      0.95      1929
         1.0       0.07      0.15      0.10        71

   micro avg       0.90      0.90      0.90      2000
   macro avg       0.52      0.54      0.52      2000
weighted avg       0.94      0.90      0.92      2000



In [92]:
predictions_forNULL = pipeline.predict(Description2)

In [103]:
p2['PreventiveFlag'][10000:] = (pd.Series(predictions_forNULL))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [105]:
p2.to_csv('new_p2.csv')